In [ ]:
# /content/drive/MyDrive/11111

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
!pip install deepface                       
import cv2
import numpy as np
from keras.models import model_from_json
from google.colab.patches import cv2_imshow
import imutils
import argparse
from deepface import DeepFace
import os
import uuid
import pandas as pd
import datetime

import tensorflow as tf

import glob
from tqdm._tqdm_notebook import tqdm_notebook as tqdm

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Convolution2D, Dropout, Dense,MaxPooling2D
from keras.layers import BatchNormalization
from keras.layers import MaxPooling2D
from keras.layers import Flatten

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D

from keras.models import Model
from tensorflow.keras.models import Model

import matplotlib.pyplot as plt
from keras.applications import vgg16

Mounted at /content/drive


In [ ]:
# pass here your video path
# you may download one from here : https://www.pexels.com/video/three-girls-laughing-5273028/
cap = cv2.VideoCapture("/content/drive/MyDrive/11111/trim_video_2.mp4")
#cap.set(cv2.CAP_PROP_FPS, 2.0)

# get width and height
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH )
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT )


# count the number of frames
frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
fps = cap.get(cv2.CAP_PROP_FPS)

  
# calculate duration of the video
seconds = round(frames / fps)
video_time = datetime.timedelta(seconds=seconds)
print(f"Number of frames in the video: {frames}, Fps :{fps}, Duration : {video_time} width: {width} height : {height}")

Number of frames in the video: 51.0, Fps :25.0, Duration : 0:00:02 width: 1280.0 height : 720.0


In [ ]:
# choose codec according to format needed
#fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
#video = cv2.VideoWriter('video.avi', fourcc, 1, (640, 480))

In [ ]:
# Define a list to store the face embeddings of all unique faces seen in the video
whole_face_embeddings = []
seen_face_embeddings = []
seen_face_id = []
final_data=[]
threshold = 0.8
models = ["VGG-Face", "Facenet", "OpenFace", "DeepFace", "DeepID", "Dlib", "ArcFace"]
frame_img_array = []

suspecious_model = tf.keras.models.load_model('/content/drive/MyDrive/11111/my_model.h5')



def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))



In [ ]:
#cap = cv2.VideoCapture(...) # open a video file or camera
frame_no=0
person_id=0
assert cap.isOpened(), "file/camera could not be opened!"
while True:
   (success, img) = cap.read() # cap.read() always returns a tuple of two things
   #img = cv2.resize(img, (720, 600))
   if not success: 
     break # you absolutely must check this
   else:
      #print("Frame_no",str(frame_no))
      frame_no=int(frame_no)+int(1)

      #cv2_imshow(img)
      #cv2.imshow("Video", img)
      try:
          objs = DeepFace.analyze(img, actions = ['age', 'gender', 'race', 'emotion'])
          for obj in objs:
            #print(obj)
            x,y,w,h=obj['region']['x'],obj['region']['y'],obj['region']['w'],obj['region']['h']
            #print(obj['region'])
            gender=obj['dominant_gender']
            emotion=obj['dominant_emotion']
            race=obj['dominant_race']
            age=obj['age']

            # Crop the face from the image
            cropped_img = img[y:y + h, x:x + w]

            resize_image = cv2.resize(cropped_img, (224, 224))
            test_array = []
            test_array.append(resize_image)
            test_single_image = np.array(test_array)
            prob_suspecious = suspecious_model.predict(test_single_image) 
            prob_class = int(prob_suspecious.argmax(axis=-1)[0])
            print(prob_class)


            embedding=DeepFace.represent(cropped_img, model_name = models[1], enforce_detection = False)
            face_embedding = embedding[0]['embedding']
            whole_face_embeddings.append(face_embedding)
            


            # Check if the face embedding is similar to any of the seen face embeddings
            similar = False
            for seen_embedding in seen_face_embeddings:
                cos_distance=cosine_similarity(np.array(face_embedding), np.array(seen_embedding))
                #print('cosine_distance',cos_distance)

                # Set a threshold for the distance, below which the two face embeddings are considered similar
                if cos_distance > threshold:
                    similar = True
                    break

            # If the face embedding is not similar to any face embeddings we have
            # seen so far, add it to the list of seen face embeddings
            if not similar:
                  #user_id=uuid.uuid4()
                  seen_face_id.append(person_id)
                  seen_face_embeddings.append(face_embedding)
                  data=[person_id,x,y,w,h,gender,emotion,race,age,frame_no,prob_class]
                  final_data.append(data)
                  person_id=int(person_id)+int(1)

          for index,seen_embedding in enumerate(seen_face_embeddings,start=0):
            cos_distance=cosine_similarity(np.array(face_embedding), np.array(seen_embedding))
            if cos_distance > 0.8:
              id=seen_face_id[index]
              data=[id,x,y,w,h,gender,emotion,race,age,frame_no,prob_class]
              final_data.append(data)
            

            cv2.rectangle(img, (x, y-50), (x+w, y+h+10), (0, 255, 0), 4)
            cv2.putText(img, f"{obj['dominant_gender']},{obj['dominant_emotion']},{obj['age']}", (x+5, y-20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
          frame_img_array.append(img)
         
          #cv2_imshow(img)
      except Exception as e:
          print(str(e))
      
   if cv2.waitKey(1) & 0xFF == ord('q'):
      break
cap.release()

cv2.destroyAllWindows()
# Print the number of unique faces in the video
print("Number of unique faces in the video: ", len(seen_face_embeddings))
print("Number of unique faces ID in the video: ", len(seen_face_id))
print("Number of total embedding: ", len(whole_face_embeddings))
#print(seen_face_embeddings)
# Create the pandas DataFrame
df = pd.DataFrame(final_data, columns=['Face ID', 'X','Y','W','H','Gender','Emotion','Race','Age','Frame No','Suspecious'])





Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/facial_expression_model_weights.h5
To: /root/.deepface/weights/facial_expression_model_weights.h5


facial_expression_model_weights.h5 will be downloaded...


100%|██████████| 5.98M/5.98M [00:00<00:00, 103MB/s]


age_model_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/age_model_weights.h5
To: /root/.deepface/weights/age_model_weights.h5
100%|██████████| 539M/539M [00:06<00:00, 87.4MB/s]


...

1/1 [==============================] - 0s 475ms/step
0
1/1 [==============================] - 0s 491ms/step
0
1/1 [==============================] - 0s 488ms/step
0


Action: emotion: 100%|██████████| 4/4 [00:01<00:00,  2.25it/s]


1/1 [==============================] - 0s 482ms/step
0
1/1 [==============================] - 0s 488ms/step
0
1/1 [==============================] - 0s 492ms/step
0


Action: emotion: 100%|██████████| 4/4 [00:02<00:00,  1.48it/s]


1/1 [==============================] - 1s 901ms/step
0
1/1 [==============================] - 0s 482ms/step
0
1/1 [==============================] - 0s 486ms/step
0
Number of unique faces in the video:  6
Number of unique faces ID in the video:  6
Number of total embedding:  156


In [ ]:
len(frame_img_array)

51

In [ ]:
#process_folder='/content/drive/MyDrive/Colab Notebooks/process/'

#for i in range(len(frame_img_array)):
#    cv2.imwrite(f'{process_folder}{i}.png',frame_img_array[i])


In [ ]:
!pip install sk-video

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 59.6 MB/s eta 0:00:00


In [ ]:
import numpy as np
import skvideo.io

#libx264
output_params={'-vcodec': 'libx264', '-crf':'15','-preset': 'ultrafast', '-pix_fmt': 'yuv444p','-r':'16'}   
out_video =  np.empty([len(frame_img_array), int(height), int(width), 3], dtype = np.uint8)
out_video =  out_video.astype(np.uint8)
  
for i in range(len(frame_img_array)):
    #cv2_imshow(frame_img_array[i])
    #img = cv2.imread(str(i) + '.png')
    rgb_image = cv2.cvtColor(frame_img_array[i], cv2.COLOR_BGR2RGB)
    out_video[i] = rgb_image

# Writes the the output image sequences in a video file
skvideo.io.vwrite("suman.mp4", out_video,outputdict=output_params)

In [ ]:
df.head()

,Face ID,X,Y,W,H,Gender,Emotion,Race,Age,Frame No,Suspecious
0,0,948,50,121,121,Man,sad,middle eastern,26,1,1
1,0,620,174,102,102,Woman,happy,white,28,1,0
2,1,285,207,203,203,Man,fear,middle eastern,28,2,1
3,1,285,207,203,203,Man,fear,middle eastern,28,2,1
4,0,266,213,205,205,Man,neutral,latino hispanic,27,3,1


In [ ]:
len(df)

71

In [ ]:
df["Suspecious"].value_counts()

0    60
1    11
Name: Suspecious, dtype: int64

In [ ]:
df[df["Face ID"]==0]['Suspecious'].value_counts()

0    19
1     9
Name: Suspecious, dtype: int64

In [ ]:
df.to_csv('/content/drive/MyDrive/11111/video.csv', index=False)

In [ ]:
df_csv = pd.read_csv ('/content/drive/MyDrive/11111/video.csv')
df_csv.head()

,Face ID,X,Y,W,H,Gender,Emotion,Race,Age,Frame No,Suspecious
0,0,948,50,121,121,Man,sad,middle eastern,26,1,1
1,0,620,174,102,102,Woman,happy,white,28,1,0
2,1,285,207,203,203,Man,fear,middle eastern,28,2,1
3,1,285,207,203,203,Man,fear,middle eastern,28,2,1
4,0,266,213,205,205,Man,neutral,latino hispanic,27,3,1


In [ ]:
df_csv.loc[df_csv['Face ID'] == 0]

,Face ID,X,Y,W,H,Gender,Emotion,Race,Age,Frame No,Suspecious
0,0,948,50,121,121,Man,sad,middle eastern,26,1,1
1,0,620,174,102,102,Woman,happy,white,28,1,0
4,0,266,213,205,205,Man,neutral,latino hispanic,27,3,1
5,0,1188,496,67,67,Man,neutral,white,26,4,1
6,0,209,194,228,228,Man,neutral,latino hispanic,32,5,0
7,0,194,189,222,222,Man,angry,latino hispanic,31,6,0
12,0,102,421,101,101,Man,neutral,white,24,10,1
26,0,598,8,281,281,Man,neutral,indian,25,23,0
28,0,1013,58,130,130,Man,angry,middle eastern,25,25,0
32,0,201,170,265,265,Man,neutral,indian,33,29,0


In [ ]:
df_csv["Face ID"].value_counts()

0    28
2    17
3    12
4    11
1     2
5     1
Name: Face ID, dtype: int64

In [ ]:
new_df = df_csv["Face ID"].value_counts().rename_axis('index').to_frame('Frame Count')
new_df['Face ID'] = new_df.index
new_df.reset_index(drop=True, inplace=True)
print (new_df)

   Frame Count  Face ID
0           28        0
1           17        2
2           12        3
3           11        4
4            2        1
5            1        5


In [ ]:
new_df['Time']=new_df['Frame Count']/fps
new_df

,Frame Count,Face ID,Time
0,28,0,1.12
1,17,2,0.68
2,12,3,0.48
3,11,4,0.44
4,2,1,0.08
5,1,5,0.04


In [ ]:
df_csv[df_csv["Face ID"]==5].Gender.value_counts()

Man    1
Name: Gender, dtype: int64

In [ ]:
df_csv

,Face ID,X,Y,W,H,Gender,Emotion,Race,Age,Frame No,Suspecious
0,0,948,50,121,121,Man,sad,middle eastern,26,1,1
1,0,620,174,102,102,Woman,happy,white,28,1,0
2,1,285,207,203,203,Man,fear,middle eastern,28,2,1
3,1,285,207,203,203,Man,fear,middle eastern,28,2,1
4,0,266,213,205,205,Man,neutral,latino hispanic,27,3,1
...,...,...,...,...,...,...,...,...,...,...,...
66,5,810,63,160,160,Man,neutral,indian,31,50,0
67,0,347,126,275,275,Man,neutral,indian,34,50,0
68,4,347,126,275,275,Man,neutral,indian,34,50,0
69,0,352,111,282,282,Man,neutral,indian,33,51,0


In [ ]:
thresold_suspecious_count=5

def get_mode(id,col):
  #print(df_csv[df_csv["Face ID"]==id]['Gender'].value_counts().index.tolist()[0])
  return df_csv[df_csv["Face ID"]==id][col].value_counts().index.tolist()[0]

def get_suspecious(id,col):
  values = df_csv[df_csv["Face ID"]==id]['Suspecious'].value_counts(dropna=False).keys().tolist()
  counts = df_csv[df_csv["Face ID"]==id]['Suspecious'].value_counts(dropna=False).tolist()
  value_dict = dict(zip(values, counts))
  #print(value_dict)
  if len(value_dict)>1:
        if value_dict[1]>thresold_suspecious_count:
          return 1
        else:
          return 0
  else:
        return 0

  #return df_csv[df_csv["Face ID"]==id][col].value_counts().index.tolist() 
   

# Because remember `apply` takes a function that gets a row (or column) passed to it
new_df["Gender"] = new_df.apply(lambda row: get_mode(int(row["Face ID"]),"Gender"),axis=1)
new_df["Emotion"] = new_df.apply(lambda row: get_mode(int(row["Face ID"]),"Emotion"),axis=1)
new_df["Race"] = new_df.apply(lambda row: get_mode(int(row["Face ID"]),"Race"),axis=1)
new_df["Age"] = new_df.apply(lambda row: get_mode(int(row["Face ID"]),"Age"),axis=1)
new_df["Suspecious"] = new_df.apply(lambda row: get_suspecious(int(row["Face ID"]),"Suspecious"),axis=1)

In [ ]:
new_df

,Frame Count,Face ID,Time,Gender,Emotion,Race,Age,Suspecious
0,28,0,1.12,Man,neutral,indian,33,1
1,17,2,0.68,Man,angry,indian,33,0
2,12,3,0.48,Man,neutral,indian,31,0
3,11,4,0.44,Man,neutral,indian,35,0
4,2,1,0.08,Man,fear,middle eastern,28,0
5,1,5,0.04,Man,neutral,indian,31,0


In [ ]:
df.to_csv('/content/drive/MyDrive/11111/processed.csv', index=False)

## Starting Model Trainning from here

In [ ]:
## for data preparation
folder='/content/drive/MyDrive/11111/suspecious'
i=0
for filename in os.listdir(folder):
    img = cv2.imread(os.path.join(folder,filename))
    if img is not None:
        try:
          objs = DeepFace.analyze(img)
          for obj in objs:
            #print(obj)
            x,y,w,h=obj['region']['x'],obj['region']['y'],obj['region']['w'],obj['region']['h']
            # Crop the face from the image
            cropped_img = img[y:y + h, x:x + w]
            resize_image = cv2.resize(cropped_img, (224, 224))
            cv2.imwrite(f'/content/drive/MyDrive/11111/dataset/suspecious/{i}.jpg',resize_image)
            i+=1
            #cv2_imshow(cropped_img)
            
        except Exception as e:
          print(str(e))
      
     

Action: race: 100%|██████████| 4/4 [00:01<00:00,  2.06it/s]


Face could not be detected. Please confirm that the picture is a face photo or consider to set enforce_detection param to False.


Action: race: 100%|██████████| 4/4 [00:01<00:00,  2.20it/s]


Face could not be detected. Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
Face could not be detected. Please confirm that the picture is a face photo or consider to set enforce_detection param to False.


Action: race: 100%|██████████| 4/4 [00:01<00:00,  2.25it/s]


...

Face could not be detected. Please confirm that the picture is a face photo or consider to set enforce_detection param to False.


Action: race: 100%|██████████| 4/4 [00:02<00:00,  1.59it/s]


Face could not be detected. Please confirm that the picture is a face photo or consider to set enforce_detection param to False.


Action: race: 100%|██████████| 4/4 [00:02<00:00,  1.81it/s]


Face could not be detected. Please confirm that the picture is a face photo or consider to set enforce_detection param to False.


In [ ]:
import tensorflow as tf

import glob
from tqdm._tqdm_notebook import tqdm_notebook as tqdm

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Convolution2D, Dropout, Dense,MaxPooling2D
from keras.layers import BatchNormalization
from keras.layers import MaxPooling2D
from keras.layers import Flatten

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D

from keras.models import Model
from tensorflow.keras.models import Model

import matplotlib.pyplot as plt
from keras.applications import vgg16

In [ ]:
X = []
y = []
folder_nonsuspecious='/content/drive/MyDrive/11111/dataset/nonsuspecious'
folder_suspecious='/content/drive/MyDrive/11111/dataset/suspecious'


In [ ]:

for filename in os.listdir(folder_nonsuspecious):
    img = cv2.imread(os.path.join(folder_nonsuspecious,filename))
    img = cv2.resize(img,(224,224))
    X.append(img)
    y.append('N')

    #vertical flip
    img_flip_ud = cv2.flip(img, 0)
    X.append(img_flip_ud)
    y.append('N')

    #horizontal flip
    img_flip_lr = cv2.flip(img, 1)
    X.append(img_flip_lr)
    y.append('N')

    #ROTATE_90_CLOCKWISE
    img_rotate_90_clockwise = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
    X.append(img_rotate_90_clockwise)
    y.append('N')

    #ROTATE_90_COUNTERCLOCKWISE
    img_rotate_90_counterclockwise = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)
    X.append(img_rotate_90_counterclockwise)
    y.append('N')

    #ROTATE_180
    img_rotate_180 = cv2.rotate(img, cv2.ROTATE_180)
    X.append(img_rotate_180)
    y.append('N')

    #gaussian_blur
    img_gaussian_blur = cv2.GaussianBlur(img,(5,5),0)
    X.append(img_gaussian_blur)
    y.append('N')

print(f'X : {len(X)}, y : {len(y)}')    

X : 245, y : 245


In [ ]:
for filename in os.listdir(folder_suspecious):
    #print(os.path.join(folder_suspecious,filename))
    img = cv2.imread(os.path.join(folder_suspecious,filename))
    #print(img.shape)
    #cv2_imshow(img)
    img = cv2.resize(img,(224,224))
    X.append(img)
    y.append('Y')

    #vertical flip
    img_flip_ud = cv2.flip(img, 0)
    X.append(img_flip_ud)
    y.append('Y')

    #horizontal flip
    img_flip_lr = cv2.flip(img, 1)
    X.append(img_flip_lr)
    y.append('Y')

    #ROTATE_90_CLOCKWISE
    img_rotate_90_clockwise = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
    X.append(img_rotate_90_clockwise)
    y.append('Y')

    #ROTATE_90_COUNTERCLOCKWISE
    img_rotate_90_counterclockwise = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)
    X.append(img_rotate_90_counterclockwise)
    y.append('Y')

    #ROTATE_180
    img_rotate_180 = cv2.rotate(img, cv2.ROTATE_180)
    X.append(img_rotate_180)
    y.append('Y')

    #gaussian_blur
    img_gaussian_blur = cv2.GaussianBlur(img,(5,5),0)
    X.append(img_gaussian_blur)
    y.append('Y')
print(f'X : {len(X)}, y : {len(y)}')  

X : 721, y : 721


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)
print ("Shape of an image in X_train: ", X_train[0].shape)
print ("Shape of an image in X_test: ", X_test[0].shape)

Shape of an image in X_train:  (224, 224, 3)
Shape of an image in X_test:  (224, 224, 3)


In [ ]:
le = preprocessing.LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)
y_train = tf.keras.utils.to_categorical(y_train, num_classes=2)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=2)
y_train = np.array(y_train)
X_train = np.array(X_train)
y_test = np.array(y_test)
X_test = np.array(X_test) 
print("X_train Shape: ", X_train.shape) 
print("X_test Shape: ", X_test.shape)
print("y_train Shape: ", y_train.shape) 
print("y_test Shape: ", y_test.shape)

X_train Shape:  (648, 224, 224, 3)
X_test Shape:  (73, 224, 224, 3)
y_train Shape:  (648, 2)
y_test Shape:  (73, 2)


In [ ]:



img_rows, img_cols = 224, 224 


vgg = vgg16.VGG16(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows, img_cols, 3))

# Here we freeze the last 4 layers 
# Layers are set to trainable as True by default
for layer in vgg.layers:
    layer.trainable = False

# Let's print our layers 
for (i,layer) in enumerate(vgg.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)    

58889256/58889256 [==============================] - 0s 0us/step
0 InputLayer False
1 Conv2D False
2 Conv2D False
3 MaxPooling2D False
4 Conv2D False
5 Conv2D False
6 MaxPooling2D False
7 Conv2D False
8 Conv2D False
9 Conv2D False
10 MaxPooling2D False
11 Conv2D False
12 Conv2D False
13 Conv2D False
14 MaxPooling2D False
15 Conv2D False
16 Conv2D False
17 Conv2D False
18 MaxPooling2D False


In [ ]:
def lw(bottom_model, num_classes):
    """creates the top or head of the model that will be 
    placed ontop of the bottom layers"""

    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    op_model = BatchNormalization()(top_model)

    top_model = Dense(1024,activation='relu')(top_model)
    top_model = BatchNormalization()(top_model)

    top_model = Dense(1024,activation='relu')(top_model)
    top_model = BatchNormalization()(top_model)

    top_model = Dense(512,activation='relu')(top_model)
    top_model = BatchNormalization()(top_model)
    
    top_model = Dense(num_classes,activation='sigmoid')(top_model)
    return top_model 

In [ ]:
num_classes = 2

FC_Head = lw(vgg, num_classes)

model = Model(inputs = vgg.input, outputs = FC_Head)

print(model.summary())

model.compile(optimizer='adam', loss = 'binary_crossentropy',metrics = ['accuracy'])
history = model.fit(X_train,y_train,
                    epochs=30, 
                    validation_data=(X_test,y_test),
                    verbose = 1,
                    initial_epoch=0)

In [ ]:
%matplotlib inline
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()

plt.show()

In [ ]:
model.save('/content/drive/MyDrive/Colab Notebooks/my_model_2.h5')

In [ ]:
new_model = tf.keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/my_model_2.h5')

# Check its architecture
new_model.summary()

In [ ]:
new_model = tf.keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/my_model_2.h5')

In [ ]:
img = cv2.imread('/content/drive/MyDrive/11111/dataset/nonsuspecious/15.jpg')
cv2_imshow(img) 
test_array = []
test_array.append(img)
test_single_image = np.array(test_array)
print("test_single_image Shape: ", test_single_image.shape) 

In [ ]:
print(new_model.predict(test_single_image))


In [ ]:
y_prob = new_model.predict(test_single_image) 
y_classes = y_prob.argmax(axis=-1)
print(int(y_classes[0]))